In [ ]:
# ============================
# CELL 1: Environment Setup
# ============================

# Core
import numpy as np
import pandas as pd
import random

# ML
import torch
import torch.nn as nn
import torch.optim as optim

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Explainability
from captum.attr import IntegratedGradients

# ----------------------------
# Reproducibility
# ----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ----------------------------
# Device
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Environment ready ✅")
print("Torch version:", torch.__version__)
print("Using device:", device)

# Verify Captum
try:
    _ = IntegratedGradients
    print("Captum available ✅")
except:
    print("Captum NOT available ❌")


Environment ready ✅
Torch version: 2.9.0+cpu
Using device: cpu
Captum available ✅


In [ ]:
!pip install captum

In [ ]:
# ============================
# CELL 2: Load Rossmann Data
# ============================

# Update paths if needed (Colab default: /content/)
TRAIN_PATH = "/content/train.csv"
TEST_PATH  = "/content/test.csv"
STORE_PATH = "/content/store.csv"

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)
store_df = pd.read_csv(STORE_PATH)

print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)
print("Store shape:", store_df.shape)

print("\n--- Train columns ---")
print(train_df.columns.tolist())

print("\n--- Sample rows (train) ---")
display(train_df.head())

# ----------------------------
# Basic sanity checks
# ----------------------------
print("\nMissing values (train):")
print(train_df.isnull().sum().sort_values(ascending=False).head(10))

print("\nDate range:")
print("Min date:", train_df['Date'].min())
print("Max date:", train_df['Date'].max())

print("\nUnique stores:", train_df['Store'].nunique())

# ----------------------------
# Merge store info (no leakage)
# ----------------------------
train_merged = train_df.merge(store_df, on="Store", how="left")
test_merged  = test_df.merge(store_df, on="Store", how="left")

print("\nAfter merge:")
print("Train merged shape:", train_merged.shape)
print("Test merged shape :", test_merged.shape)

# ----------------------------
# Target distribution
# ----------------------------
print("\nSales stats:")
print(train_merged['Sales'].describe())


/tmp/ipython-input-3502246302.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(TRAIN_PATH)


Train shape: (1017209, 9)
Test shape : (41088, 8)
Store shape: (1115, 10)

--- Train columns ---
['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']

--- Sample rows (train) ---


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1



Missing values (train):
Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Date range:
Min date: 2013-01-01
Max date: 2015-07-31

Unique stores: 1115

After merge:
Train merged shape: (1017209, 18)
Test merged shape : (41088, 17)

Sales stats:
count    1.017209e+06
mean     5.773819e+03
std      3.849926e+03
min      0.000000e+00
25%      3.727000e+03
50%      5.744000e+03
75%      7.856000e+03
max      4.155100e+04
Name: Sales, dtype: float64


In [ ]:
# =============================
# CELL 3: Feature Engineering
# =============================

import numpy as np
from sklearn.model_selection import train_test_split

df = train_merged.copy()

# -----------------------------
# Date features
# -----------------------------
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# -----------------------------
# Categorical cleanup
# -----------------------------
df['StateHoliday'] = df['StateHoliday'].astype(str)

# -----------------------------
# Target transform (standard in literature)
# -----------------------------
df = df[df['Open'] == 1]  # closed stores → zero sales (remove)
df['LogSales'] = np.log1p(df['Sales'])

# -----------------------------
# Feature selection (baseline-safe)
# -----------------------------
features = [
    'Store', 'DayOfWeek', 'Promo', 'Promo2',
    'SchoolHoliday', 'Customers',
    'Year', 'Month', 'Day', 'WeekOfYear',
    'StoreType', 'Assortment', 'CompetitionDistance'
]

X = df[features]
y = df['LogSales']

# -----------------------------
# Train / validation split (random, standard)
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)

print("\nTarget stats (log-sales):")
print(y_train.describe())


Train size: (675513, 13)
Validation size: (168879, 13)

Target stats (log-sales):
count    675513.000000
mean          8.757269
std           0.430584
min           0.000000
25%           8.488794
50%           8.759512
75%           9.031333
max          10.634701
Name: LogSales, dtype: float64


In [ ]:
# =============================
# CELL 4: Baseline 1 — Linear Regression (with imputation)
# =============================

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer

# -----------------------------
# Feature types
# -----------------------------
categorical_features = ['StoreType', 'Assortment']
numeric_features = [c for c in X_train.columns if c not in categorical_features]

# -----------------------------
# Preprocessing with imputation
# -----------------------------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# -----------------------------
# Linear model (Ridge)
# -----------------------------
linreg = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', Ridge(alpha=1.0))
])

# -----------------------------
# Train
# -----------------------------
linreg.fit(X_train, y_train)

# -----------------------------
# Evaluate
# -----------------------------
# -----------------------------
# Evaluate
# -----------------------------
y_pred = linreg.predict(X_val)

# RMSE manually
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print("Linear Regression (Ridge) Results with Imputation")
print(f"RMSE (log-sales): {rmse:.4f}")
print(f"MAE  (log-sales): {mae:.4f}")


Linear Regression (Ridge) Results with Imputation
RMSE (log-sales): 0.2246
MAE  (log-sales): 0.1617


In [ ]:
# =============================
# CELL 5: Baseline 2 — Random Forest
# =============================

from sklearn.ensemble import RandomForestRegressor

# -----------------------------
# Random Forest (natively handles nonlinearities)
# -----------------------------
rf_model = Pipeline(steps=[
    ('preprocess', preprocessor),  # same imputation + scaling + one-hot
    ('model', RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        n_jobs=-1,
        random_state=42
    ))
])

# -----------------------------
# Train
# -----------------------------
rf_model.fit(X_train, y_train)

# -----------------------------
# Evaluate
# -----------------------------
y_pred_rf = rf_model.predict(X_val)

rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
mae_rf = mean_absolute_error(y_val, y_pred_rf)

print("Random Forest Results")
print(f"RMSE (log-sales): {rmse_rf:.4f}")
print(f"MAE  (log-sales): {mae_rf:.4f}")


Random Forest Results
RMSE (log-sales): 0.0919
MAE  (log-sales): 0.0695


In [ ]:
# =============================
# CELL 6 (FIXED): LightGBM Baseline with Encoding
# =============================

import numpy as np
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -----------------------------
# Identify categorical columns
# -----------------------------
categorical_cols = ['StoreType', 'Assortment']
numeric_cols = [c for c in X_train.columns if c not in categorical_cols]

# -----------------------------
# Preprocessing
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# -----------------------------
# LightGBM model
# -----------------------------
lgbm_model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=10,
    num_leaves=31,
    random_state=42
)

# -----------------------------
# Pipeline
# -----------------------------
pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', lgbm_model)
])

# -----------------------------
# Train
# -----------------------------
pipeline.fit(X_train, y_train)

# -----------------------------
# Predict & Evaluate
# -----------------------------
y_pred_gbm = pipeline.predict(X_val)

rmse_gbm = np.sqrt(mean_squared_error(y_val, y_pred_gbm))
mae_gbm = mean_absolute_error(y_val, y_pred_gbm)

print("LightGBM Results (Encoded)")
print(f"RMSE (log-sales): {rmse_gbm:.4f}")
print(f"MAE  (log-sales): {mae_gbm:.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029995 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 894
[LightGBM] [Info] Number of data points in the train set: 675513, number of used features: 18
[LightGBM] [Info] Start training from score 8.757269


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


LightGBM Results (Encoded)
RMSE (log-sales): 0.0682
MAE  (log-sales): 0.0512


In [8]:
# =============================
# CELL: Hierarchical Neural Network (NaN-safe, Rossmann)
# =============================

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -----------------------------
# 1. FILTER + SAFE TARGET
# -----------------------------
# Assumes df already exists (train + store merged)

df_nn = df.copy()

# Remove closed stores and zero sales (CRITICAL)
df_nn = df_nn[df_nn["Open"] == 1]
df_nn = df_nn[df_nn["Sales"] > 0]

# Safe log transform
df_nn["LogSales"] = np.log1p(df_nn["Sales"])

# -----------------------------
# 2. FEATURE ENGINEERING
# -----------------------------
df_nn["Date"] = pd.to_datetime(df_nn["Date"])
df_nn["Month"] = df_nn["Date"].dt.month

# Business-aligned features
control_features = ["Promo", "Promo2"]
context_features = ["DayOfWeek", "Customers", "SchoolHoliday", "Month"]
structural_features = ["CompetitionDistance", "StoreType", "Assortment"]

features = control_features + context_features + structural_features
target = "LogSales"

X = df_nn[features]
y = df_nn[target]

# -----------------------------
# 3. IMPUTE NUMERICS
# -----------------------------
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())

# -----------------------------
# 4. ONE-HOT ENCODE CATEGORICALS
# -----------------------------
X = pd.get_dummies(X, columns=["StoreType", "Assortment"], drop_first=True)

# -----------------------------
# 5. TRAIN / VALIDATION SPLIT
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 6. SCALE FEATURES
# -----------------------------
scaler = StandardScaler()
X_train = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_val = pd.DataFrame(
    scaler.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)

# -----------------------------
# 7. FEATURE GROUP SLICES
# -----------------------------
all_features = list(X_train.columns)

control_idx = [all_features.index(f) for f in control_features]
context_idx = [all_features.index(f) for f in context_features]
structure_idx = [
    i for i, c in enumerate(all_features)
    if c.startswith("CompetitionDistance")
    or c.startswith("StoreType_")
    or c.startswith("Assortment_")
]

slices = {
    "Control": control_idx,
    "Context": context_idx,
    "Structure": structure_idx
}

print("Feature group sizes:")
for k, v in slices.items():
    print(f"{k}: {len(v)} features")

# -----------------------------
# 8. TENSORS
# -----------------------------
X_train_t = torch.tensor(X_train.values, dtype=torch.float32)
X_val_t   = torch.tensor(X_val.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_val_t   = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 9. HIERARCHICAL NN
# -----------------------------
class HierarchicalNN(nn.Module):
    def __init__(self, slices):
        super().__init__()
        H = 64

        self.control_fc = nn.Sequential(
            nn.Linear(len(slices["Control"]), H),
            nn.ReLU()
        )

        self.context_fc = nn.Sequential(
            nn.Linear(H + len(slices["Context"]), H),
            nn.ReLU()
        )

        self.structure_fc = nn.Sequential(
            nn.Linear(H + len(slices["Structure"]), H),
            nn.ReLU()
        )

        self.out = nn.Linear(H, 1)

    def forward(self, x):
        c = x[:, slices["Control"]]
        ctx = x[:, slices["Context"]]
        s = x[:, slices["Structure"]]

        h_c = self.control_fc(c)
        h_ctx = self.context_fc(torch.cat([h_c, ctx], dim=1))
        h_s = self.structure_fc(torch.cat([h_ctx, s], dim=1))

        return self.out(h_s)

# -----------------------------
# 10. TRAINING
# -----------------------------
model = HierarchicalNN(slices).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

train_loader = DataLoader(
    TensorDataset(X_train_t.to(device), y_train_t.to(device)),
    batch_size=2048,
    shuffle=True
)

EPOCHS = 30

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()

        # CRITICAL: gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch [{epoch}/{EPOCHS}] | Train MSE: {total_loss / len(train_loader.dataset):.4f}")

print("Hierarchical NN training complete ✅")

# -----------------------------
# 11. VALIDATION
# -----------------------------
model.eval()
with torch.no_grad():
    val_preds = model(X_val_t.to(device)).cpu().numpy()

rmse_nn = np.sqrt(mean_squared_error(y_val, val_preds))
mae_nn  = mean_absolute_error(y_val, val_preds)

print("\nHierarchical NN Results")
print(f"RMSE (log-sales): {rmse_nn:.4f}")
print(f"MAE  (log-sales): {mae_nn:.4f}")


/tmp/ipython-input-3285588649.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())


Feature group sizes:
Control: 2 features
Context: 4 features
Structure: 6 features
Epoch [1/30] | Train MSE: 5.1891
Epoch [5/30] | Train MSE: 0.0240
Epoch [10/30] | Train MSE: 0.0225
Epoch [15/30] | Train MSE: 0.0229
Epoch [20/30] | Train MSE: 0.0223
Epoch [25/30] | Train MSE: 0.0217
Epoch [30/30] | Train MSE: 0.0221
Hierarchical NN training complete ✅

Hierarchical NN Results
RMSE (log-sales): 0.1541
MAE  (log-sales): 0.1212


In [9]:
model.eval()
with torch.no_grad():
    val_preds = model(X_val_t.to(device)).cpu().numpy()

rmse_nn = np.sqrt(mean_squared_error(y_val, val_preds))
mae_nn  = mean_absolute_error(y_val, val_preds)

print(f"Hierarchical NN RMSE: {rmse_nn:.4f}")
print(f"Hierarchical NN MAE : {mae_nn:.4f}")


Hierarchical NN RMSE: 0.1541
Hierarchical NN MAE : 0.1212


In [20]:
# =========================================
# FINAL CELL: Business-Aware GNN + Captum Explainability
# =========================================

!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -q
!pip install captum -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from captum.attr import IntegratedGradients
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import pandas as pd

# -----------------------------
# 0. Seed + device
# -----------------------------
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 1. Simulated business data
# -----------------------------
def generate_business_data(n_samples=6000):
    rng = np.random.RandomState(RANDOM_SEED)
    month = rng.randint(1,13,n_samples)
    seasonality = 12 * np.sin(month*np.pi/6)
    sales_lag1 = rng.normal(100,25,n_samples).clip(20)
    cannibal_sales_lag1 = rng.normal(60,18,n_samples).clip(10)
    outlet_type = rng.choice(["Premium","Standard","Value"], n_samples, p=[0.25,0.55,0.20])
    brand_equity = rng.uniform(0.7,1.0,n_samples)
    ad_spend = rng.choice([0,500,1000,2000], n_samples, p=[0.45,0.30,0.15,0.10])
    promo_type = rng.choice(["None","Discount","Bundle","Slab"], n_samples, p=[0.6,0.2,0.1,0.1])
    discount_pct = np.where(promo_type=="Discount", rng.uniform(0.1,0.35,n_samples), 0.0)
    base_map = {"Premium":95,"Standard":65,"Value":45}
    base_demand = np.vectorize(base_map.get)(outlet_type)
    momentum = 0.45*sales_lag1
    cannibal_drag = -0.18*cannibal_sales_lag1
    roas_map = {0:0,500:6,1000:14,2000:24}
    ad_lift = np.vectorize(roas_map.get)(ad_spend)
    promo_lift = np.zeros(n_samples)
    promo_lift += np.where(promo_type=="Slab",22,0)
    promo_lift += np.where(promo_type=="Bundle",15,0)
    promo_lift += np.where(promo_type=="Discount",12+90*discount_pct,0)
    brand_halo = 12*brand_equity
    noise = rng.normal(0,6,n_samples)
    units_sold = (base_demand + momentum + seasonality + ad_lift + promo_lift + cannibal_drag + brand_halo + noise).clip(5)
    df = pd.DataFrame({
        "month":month, "sales_lag1":sales_lag1, "cannibal_sales_lag1":cannibal_sales_lag1,
        "outlet_type":outlet_type, "brand_equity":brand_equity,
        "ad_spend":ad_spend, "promo_type":promo_type, "discount_pct":discount_pct,
        "units_sold":units_sold
    })
    return df

df = generate_business_data(6000)

# -----------------------------
# 2. Features & encoding
# -----------------------------
y = torch.tensor(df["units_sold"].values, dtype=torch.float32).view(-1,1)

# Contextual
context_features = ["month","sales_lag1","cannibal_sales_lag1","brand_equity"]
scaler_ctx = StandardScaler()
Xc = torch.tensor(scaler_ctx.fit_transform(df[context_features]), dtype=torch.float32)

# Control
control_numeric = ["ad_spend","discount_pct"]
scaler_ctl = StandardScaler()
Xctl_num = scaler_ctl.fit_transform(df[control_numeric])

control_categorical = ["promo_type"]
enc_promo = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
Xctl_cat = enc_promo.fit_transform(df[control_categorical])

Xctl = torch.tensor(np.concatenate([Xctl_num, Xctl_cat], axis=1), dtype=torch.float32)

# Structural
structural_features = ["outlet_type"]
enc_outlet = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
Xs = torch.tensor(enc_outlet.fit_transform(df[structural_features]), dtype=torch.float32)

# -----------------------------
# 3. Train-test split
# -----------------------------
Xc_train, Xc_test, Xctl_train, Xctl_test, Xs_train, Xs_test, y_train, y_test = train_test_split(
    Xc, Xctl, Xs, y, test_size=0.2, random_state=RANDOM_SEED
)

print("Shapes ready | Context:", Xc_train.shape, "Control:", Xctl_train.shape, "Structure:", Xs_train.shape)

# -----------------------------
# 4. Graph for structural GCN
# -----------------------------
num_nodes = Xs.shape[1]
edge_index = []
for i in range(num_nodes):
    for j in range(num_nodes):
        if i!=j:
            edge_index.append([i,j])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
print(f"Graph ready: nodes = {num_nodes} edges = {edge_index.shape[1]}")

# -----------------------------
# 5. GNN Model
# -----------------------------
class BusinessAwareGNN(nn.Module):
    def __init__(self, ctx_dim, ctl_dim, str_dim):
        super().__init__()
        H = 32
        self.context_net = nn.Sequential(nn.Linear(ctx_dim,H), nn.ReLU())
        self.control_net = nn.Sequential(nn.Linear(ctl_dim,H), nn.ReLU())
        self.structural_gcn = GCNConv(str_dim,H)
        self.fusion = nn.Sequential(nn.Linear(H*3,H), nn.ReLU(), nn.Linear(H,1))

    def forward(self, x_ctx, x_ctl, x_str):
        h_ctx = self.context_net(x_ctx)
        h_ctl = self.control_net(x_ctl)
        h_str = torch.relu(self.structural_gcn(x_str, edge_index.to(x_str.device)))
        return self.fusion(torch.cat([h_ctx,h_ctl,h_str], dim=1))

model_gnn = BusinessAwareGNN(Xc_train.shape[1], Xctl_train.shape[1], Xs_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=0.001)
criterion = nn.MSELoss()

# -----------------------------
# 6. Training loop
# -----------------------------
EPOCHS = 50
BATCH_SIZE = 64
dataset = torch.utils.data.TensorDataset(Xc_train, Xctl_train, Xs_train, y_train)
loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

for epoch in range(EPOCHS):
    model_gnn.train()
    total_loss = 0
    for xc, xctl, xs, yb in loader:
        xc, xctl, xs, yb = xc.to(device), xctl.to(device), xs.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model_gnn(xc, xctl, xs)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * yb.size(0)
    if (epoch+1)%10==0 or epoch==0:
        print(f"Epoch [{epoch+1}/{EPOCHS}] | Train MSE: {total_loss/len(dataset):.4f}")

print("GNN Training complete ✅")

# -----------------------------
# 7. Captum Integrated Gradients
# -----------------------------
model_gnn.eval()
ig = IntegratedGradients(model_gnn.forward)

baseline_ctx = torch.zeros_like(Xc_test[:1]).to(device)
baseline_ctl = torch.zeros_like(Xctl_test[:1]).to(device)
baseline_str = torch.zeros_like(Xs_test[:1]).to(device)

inputs = (Xc_test[:300].to(device), Xctl_test[:300].to(device), Xs_test[:300].to(device))
baselines = (baseline_ctx, baseline_ctl, baseline_str)

attr_ctx, attr_ctl, attr_str = ig.attribute(
    inputs=inputs,
    baselines=baselines,
    n_steps=50
)

# -----------------------------
# 8. Aggregate attribution
# -----------------------------
ctx_contrib = np.mean(np.sum(np.abs(attr_ctx.cpu().detach().numpy()),axis=1))
ctl_contrib = np.mean(np.sum(np.abs(attr_ctl.cpu().detach().numpy()),axis=1))
str_contrib = np.mean(np.sum(np.abs(attr_str.cpu().detach().numpy()),axis=1))
total = ctx_contrib + ctl_contrib + str_contrib

print("\n--- BUSINESS-ALIGNED ATTRIBUTION ---")
print(f"Context (non-actionable)   : {ctx_contrib:.2f} ({ctx_contrib/total:.1%})")
print(f"Control (business levers)  : {ctl_contrib:.2f} ({ctl_contrib/total:.1%})")
print(f"Structure (outlet effects) : {str_contrib:.2f} ({str_contrib/total:.1%})")



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires n

In [30]:
# =============================================
# FINAL SINGLE CELL: TRAIN + EVAL + ATTRIBUTION
# =============================================

import torch, torch.nn as nn, torch.optim as optim
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from captum.attr import IntegratedGradients
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ----------------------------
# Assume Xc_train/Xctl_train/Xs_train/y_train
# and Xc_test/Xctl_test/Xs_test/y_test, graph_data
# are already defined
# ----------------------------

# ----------------------------
# 1. Define GNN Model
# ----------------------------
H = 32
num_nodes = Xs_train.shape[1]  # number of structural features

class StructuralGCN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.gcn = GCNConv(in_dim, out_dim)
    def forward(self, x, edge_index):
        return torch.relu(self.gcn(x, edge_index))

class BusinessAwareGNN(nn.Module):
    def __init__(self, ctx_dim, ctl_dim, str_dim):
        super().__init__()
        self.context_net = nn.Sequential(nn.Linear(ctx_dim,H), nn.ReLU())
        self.control_net = nn.Sequential(nn.Linear(ctl_dim,H), nn.ReLU())
        self.structural_gcn = StructuralGCN(str_dim,H)
        self.fusion = nn.Sequential(nn.Linear(H*3,H), nn.ReLU(), nn.Linear(H,1))
        # TEMP fusion for IG (context + control only)
        self.fusion_ig = nn.Sequential(nn.Linear(H*2,H), nn.ReLU(), nn.Linear(H,1))
    def forward(self, x_ctx, x_ctl, x_str, graph):
        h_ctx = self.context_net(x_ctx)
        h_ctl = self.control_net(x_ctl)
        g_emb = self.structural_gcn(graph.x, graph.edge_index)
        node_ids = torch.argmax(x_str, dim=1)
        h_str = g_emb[node_ids]
        return self.fusion(torch.cat([h_ctx,h_ctl,h_str], dim=1))
    def forward_ig(self, x_ctx, x_ctl):
        h_ctx = self.context_net(x_ctx)
        h_ctl = self.control_net(x_ctl)
        return self.fusion_ig(torch.cat([h_ctx,h_ctl], dim=1))

# ----------------------------
# 2. Initialize & Train
# ----------------------------
model_gnn = BusinessAwareGNN(Xc_train.shape[1], Xctl_train.shape[1], num_nodes).to(device)
optimizer = optim.Adam(model_gnn.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
BATCH_SIZE = 64
train_dataset = torch.utils.data.TensorDataset(
    Xc_train.to(device), Xctl_train.to(device), Xs_train.to(device), y_train.to(device)
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

model_gnn.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for xc,xctl,xs,y in train_loader:
        optimizer.zero_grad()
        preds = model_gnn(xc,xctl,xs,graph_data)
        loss = criterion(preds,y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
    if (epoch+1)%10==0:
        print(f"Epoch [{epoch+1}/{EPOCHS}] | Train MSE: {total_loss/len(train_dataset):.4f}")
print("GNN Training complete ✅")

# ----------------------------
# 3. Evaluate
# ----------------------------
model_gnn.eval()
with torch.no_grad():
    preds_test = model_gnn(Xc_test.to(device), Xctl_test.to(device), Xs_test.to(device), graph_data)
rmse = torch.sqrt(torch.mean((preds_test - y_test.to(device))**2))
mae = torch.mean(torch.abs(preds_test - y_test.to(device)))
print(f"GNN RMSE: {rmse.item():.4f} | MAE: {mae.item():.4f}")

# ----------------------------
# 4. Integrated Gradients (context + control only)
# ----------------------------
N_EXPLAIN = 300
inputs_cc = torch.cat([Xc_test[:N_EXPLAIN], Xctl_test[:N_EXPLAIN]], dim=1).to(device)
baseline_cc = torch.zeros_like(inputs_cc).to(device)

def forward_cc(x):
    xc = x[:, :Xc_test.shape[1]]
    xctl = x[:, Xc_test.shape[1]:]
    return model_gnn.forward_ig(xc,xctl)

ig = IntegratedGradients(forward_cc)
attr = ig.attribute(inputs_cc, baselines=baseline_cc, n_steps=50)

attr_ctx = attr[:, :Xc_test.shape[1]].detach().cpu().numpy()
attr_ctl = attr[:, Xc_test.shape[1]:].detach().cpu().numpy()

# ----------------------------
# 5. Structural contribution (post-hoc)
# ----------------------------
with torch.no_grad():
    g_emb = model_gnn.structural_gcn(graph_data.x, graph_data.edge_index)
    node_ids = torch.argmax(Xs_test[:N_EXPLAIN], dim=1)
    attr_str = g_emb[node_ids].detach().cpu().numpy()

# ----------------------------
# 6. Aggregate
# ----------------------------
context_contribution = np.mean(np.sum(np.abs(attr_ctx), axis=1))
control_contribution = np.mean(np.sum(np.abs(attr_ctl), axis=1))
structural_contribution = np.mean(np.linalg.norm(attr_str, axis=1))
total = context_contribution + control_contribution + structural_contribution

print("\n--- BUSINESS-ALIGNED ATTRIBUTION ---")
print(f"Context   : {context_contribution:.2f} ({context_contribution/total:.1%})")
print(f"Control   : {control_contribution:.2f} ({control_contribution/total:.1%})")
print(f"Structure : {structural_contribution:.2f} ({structural_contribution/total:.1%})")


Epoch [10/50] | Train MSE: 409.9723
Epoch [20/50] | Train MSE: 365.1341
Epoch [30/50] | Train MSE: 348.0713
Epoch [40/50] | Train MSE: 340.4007
Epoch [50/50] | Train MSE: 337.1768
GNN Training complete ✅
GNN RMSE: 18.3732 | MAE: 14.5301

--- BUSINESS-ALIGNED ATTRIBUTION ---
Context   : 0.09 (2.3%)
Control   : 0.21 (5.5%)
Structure : 3.53 (92.2%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')